# 🎨 Chapter 17: Autoencoders & GANs — Practical Guide

Explore how unsupervised deep learning can learn compact representations and generate realistic data.

## I. 🤏 Efficient Data Representations

Autoencoders learn compressed, meaningful representations by training to reconstruct their inputs.

## II. 🧊 PCA with a Linear Autoencoder

Let's start with a simple linear autoencoder that learns principal components (PCA).

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Generate random data
X = np.random.rand(1000, 20)

# Define a simple linear autoencoder
auto = models.Sequential([
    layers.Dense(10, activation='linear', input_shape=(20,)),
    layers.Dense(20, activation='linear')
])

# Compile and train
auto.compile(optimizer='adam', loss='mse')
auto.fit(X, X, epochs=20, batch_size=32, verbose=2)

## III. 📚 Stacked Autoencoders

### A. Keras Implementation & Visualization

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt

# Load and preprocess data
(X_train, _), (X_test, _) = fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
X_train = X_train.reshape((-1, 784))
X_test = X_test.reshape((-1, 784))

# Define encoder and decoder layers
input_img = layers.Input(shape=(784,))
hidden1 = layers.Dense(128, activation='relu')(input_img)
hidden2 = layers.Dense(64, activation='relu')(hidden1)
hidden3 = layers.Dense(128, activation='relu')(hidden2)
output_layer = layers.Dense(784, activation='sigmoid')(hidden3)

# Build autoencoder model
stacked_ae = models.Model(inputs=input_img, outputs=output_layer)

# Compile
stacked_ae.compile(optimizer='adam', loss='binary_crossentropy')

# Train
stacked_ae.fit(X_train, X_train, epochs=10, batch_size=256, validation_data=(X_test, X_test), verbose=2)

### B. Visualize Reconstructions

Let's see how well the autoencoder reconstructs some test images.

In [ ]:
# Predict reconstructions for first 10 test images
recon = stacked_ae.predict(X_test[:10])

# Plot original and reconstructed images
plt.figure(figsize=(20,4))
for i in range(10):
    # Original
    ax = plt.subplot(2, 10, i + 1)
    plt.imshow(X_test[i].reshape(28,28), cmap='gray')
    plt.axis('off')
    # Reconstruction
    ax = plt.subplot(2,10,i+11)
    plt.imshow(recon[i].reshape(28,28), cmap='gray')
    plt.axis('off')
plt.suptitle('Top: Original | Bottom: Reconstruction')
plt.show()

## IV. 🌀 Convolutional Autoencoders

Autoencoders can also be built with convolutional layers for images.

In [ ]:
input_img = layers.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D(2, padding='same')(x)

x = layers.Conv2D(32, 3, activation='relu', padding='same')(encoded)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
decoded = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(x)

conv_ae = models.Model(inputs=input_img, outputs=decoded)

# Compile
conv_ae.compile(optimizer='adam', loss='binary_crossentropy')

# Prepare data
X_train_c = X_train.reshape(-1,28,28,1)

# Train
conv_ae.fit(X_train_c, X_train_c, epochs=10, batch_size=256, verbose=2)

## V. 🔁 Recurrent Autoencoders

Recurrent autoencoders are suitable for sequence data like text or time series. Due to brevity, code is omitted, but they follow similar structure using RNNs or LSTMs.

## VI. 💧 Denoising Autoencoders

Train autoencoders to remove noise from inputs.

In [ ]:
# Add synthetic noise to training data
noisy_X_train = X_train + np.random.normal(0, 0.5, X_train.shape)
noisy_X_train = np.clip(noisy_X_train, 0., 1.)  # Keep within valid range

# Retrain autoencoder on noisy data
auto.compile(optimizer='adam', loss='mse')  # Recompile if needed
auto.fit(noisy_X_train, X_train, epochs=10, batch_size=256, verbose=2)

## VII. ❗ Sparse Autoencoders

Encourage sparsity in hidden units with regularization.

In [ ]:
sparse_encoder = layers.Dense(64, activation='relu', activity_regularizer=tf.keras.regularizers.l1(1e-5))

# Build sparse autoencoder
input_img = layers.Input(shape=(784,))
hidden = sparse_encoder(input_img)
output_layer = layers.Dense(784, activation='sigmoid')(hidden)
sparse_autoencoder = models.Model(inputs=input_img, outputs=output_layer)

# Compile
sparse_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train
sparse_autoencoder.fit(X_train, X_train, epochs=10, batch_size=256, verbose=2)

## VIII. 🌀 Variational Autoencoders (VAEs)

VAEs learn probabilistic latent representations, enabling realistic sampling.

```python
# High-level pseudocode for VAE implementation
# 1. Encoder learns mean and log-variance of latent distribution
# 2. Sample z via reparameterization trick
# 3. Decoder reconstructs input from z
# 4. Loss includes reconstruction + KL divergence
```

## IX. ⚔️ GANs (Generative Adversarial Networks)

GANs involve training a generator and discriminator in a minimax game.

### A. Deep Convolutional GAN Example

Below is pseudocode for the typical training loop of a DCGAN.

In [ ]:
# Pseudocode for training a DCGAN
import tensorflow as tf
from tensorflow.keras import layers, models

# Define generator and discriminator models here...
# For brevity, models are not fully implemented.

# Training loop pseudocode:
for epoch in range(num_epochs):
    for batch in dataset:
        # 1. Train discriminator on real images
        # 2. Generate fake images with generator
        # 3. Train discriminator on fake images
        # 4. Train generator to fool discriminator
        pass

# Note: Implementing full DCGAN training is more involved.
# This pseudocode is a conceptual outline.

### B. Training Difficulties

* ⚠️ Mode collapse, unstable convergence
* Require careful balancing (learning rates, label smoothing, batch norm)

### C. Advanced Architectures

* **DCGAN**: Stable convolutional GANs
* **Progressive GANs**: Incrementally add layers during training
* **StyleGAN**: Style-mixing via adaptive normalization

## ✅ Chapter Summary

* **Autoencoders** learn compact, meaningful data representations.
* **Variational autoencoders** model data generatively via latent distributions.
* **GANs** generate realistic images by pitting two networks against each other.
* Advanced GANs like **StyleGAN** produce high-resolution, photorealistic images.

## 🧠 Exercises to Try

1. Implement a stacked autoencoder and measure compression vs reconstruction loss.
2. Build and train a convolutional denoising autoencoder with added synthetic noise.
3. Code a simple VAE and visualize generated Fashion MNIST samples.
4. Build a DCGAN and train it—track real vs fake image quality over epochs.
5. Try **Progressive Growing GANs** or **StyleGAN**, starting from a simpler version.